In [24]:
from transformers import ASTModel, ASTConfig
import torch
import torch.nn as nn

In [7]:
config = ASTConfig(hidden_size=384, num_attention_heads=6, intermediate_size=1536)
model = ASTModel(config=config)

config_tiny = ASTConfig(hidden_size=192, num_attention_heads=3, intermediate_size=769)
model_tiny = ASTModel(config=config)

In [12]:
input = torch.rand((1,1024,128))
output = model(input)

In [23]:
class ASTSmallDistill(nn.Module):
    
    def __init__(self, ):
    
        config = ASTConfig(hidden_size=384, num_attention_heads=6, intermediate_size=1536)
        model = ASTModel(config=config)
        pass

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, T

In [1]:
from types import SimpleNamespace

In [3]:
test = SimpleNamespace(**{"test": 5})

In [9]:
SimpleNamespace().__dict__

{}

In [1]:
from lumen_irmas import ASTPretrained

In [3]:
model = ASTPretrained(11,0.5)

Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.layernorm.bias', 'classifier.layernorm.weight', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ASTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
getattr(model.base_model, "encoder")

ASTEncoder(
  (layer): ModuleList(
    (0-11): 12 x ASTLayer(
      (attention): ASTAttention(
        (attention): ASTSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (output): ASTSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (intermediate): ASTIntermediate(
        (dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): ASTOutput(
        (dense): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm_before): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (layernorm

In [ ]:
import torch
from lumen_irmas import ASTPretrained
num_models = 2
batch_size = 4
in_features, out_features = 3, 3
models = [ASTPretrained(out_features, 0.5) for _ in range(num_models)]

In [9]:
data = torch.randn(batch_size, 1024, 128)


# ------------------------------------
# using torch.func (as of PyTorch 2.0)
# ------------------------------------
import copy

# Construct a version of the model with no memory by putting the Tensors on
# the meta device.
base_model = copy.deepcopy(models[0])
base_model.to('meta')

params, buffers = torch.func.stack_module_state(models)

# It is possible to vmap directly over torch.func.functional_call,
# but wrapping it in a function makes it clearer what is going on.
def call_single_model(params, buffers, data):
    return torch.func.functional_call(base_model, (params, buffers), (data,))

output = torch.vmap(call_single_model, (0, 0, None))(params, buffers, data)
assert output.shape == (num_models, batch_size, out_features)

: 

: 